In [5]:
import pandas as pd
from scipy import stats

pd.set_option('display.width', None)

# Ler dados
df = pd.read_csv("C:/Users/karin/OneDrive/Área de Trabalho/EBAC/clientes_limpeza.csv")

### Filtro básico de Outlier 

In [6]:
df_filtro_basico = df[df['idade'] > 100]
print('Filtro básico \n', df_filtro_basico[['nome', 'idade']])

Filtro básico 
 Empty DataFrame
Columns: [nome, idade]
Index: []


Esse filtro está buscando pessoas com idade acima de 100, mas como seu DataFrame foi limpo (com média ou dados corrigidos anteriormente), é possível que todas as idades estejam ≤ 100 — por isso o DataFrame está vazio.

##  Identificar outliers com Z-score

In [9]:
z_scores = stats.zscore(df['idade'].dropna())
outliers_z = df[z_scores >= 3]
print("Outliers pelo Z-score:\n", outliers_z)

Outliers pelo Z-score:
 Empty DataFrame
Columns: [nome, cpf, idade, data, endereco, estado]
Index: []


###  Calcular o IQR e determinar limites para outliers

In [23]:
Q1 = df['idade'].quantile(0.25)
Q3 = df['idade'].quantile(0.75)
IQR = Q3 - Q1

# 2. Determinar os limites para outliers
Limite_baixo = Q1 - 1.5 * IQR
Limite_alto = Q3 + 1.5 * IQR

print("Limites IQR:", Limite_baixo, Limite_alto)

Limites IQR: -20.5 135.5


###  Identificar os outliers pelo IQR

In [24]:
outliers_iqr = df[(df['idade'] < Limite_baixo) | (df['idade'] > Limite_alto)]
print("Outliers pelo IQR:\n", outliers_iqr)

Outliers pelo IQR:
 Empty DataFrame
Columns: [nome, cpf, idade, data, endereco, estado]
Index: []


In [25]:
# Remover os outliers do DataFrame
df = df[(df['idade'] >= Limite_baixo) & (df['idade'] <= Limite_alto)]

# Aplicar limite manual de idade entre 1 e 100 (filtro extra)
df = df[(df['idade'] >= 1) & (df['idade'] <= 100)]

### Tratar endereços inválidos (com menos de 3 palavras)

In [26]:
df['endereco'] = df['endereco'].apply(lambda x: 'Endereço inválido' if len(str(x).split()) < 3 else x)

### Tratar nomes muito grandes (mais de 50 caracteres)

In [27]:
df['nome'] = df['nome'].apply(lambda x: 'Nome inválido' if isinstance(x, str) and len(x) > 50 else x)
print('Qtd registros com nomes grandes:', (df['nome'] == 'Nome inválido').sum())

Qtd registros com nomes grandes: 0


### Tratar endereços inválidos

In [29]:
df['endereco'] = df['endereco'].apply(lambda x: 'Endereço inválido' if len(str(x).split('\n')) < 3 else x)
print('Qtd registros com endereços inválidos:', (df['endereco'] == 'Endereço inválido').sum())

Qtd registros com endereços inválidos: 975


### Tratar nomes muito longos


In [30]:
df['nome'] = df['nome'].apply(lambda x: 'Nome inválido' if isinstance(x, str) and len(x) > 50 else x)
print('Qtd registros com nomes grandes:', (df['nome'] == 'Nome inválido').sum())

Qtd registros com nomes grandes: 0


In [31]:
# Exibir resultado final
print("\nDados com Outliers tratados:\n", df)


Dados com Outliers tratados:
                         nome             cpf  idade  data           endereco  \
0    Sr. Vitor Gabriel Silva  640.***.***-01     90   NaN  Endereço inválido   
1              Calebe Santos  651.***.***-46     40   NaN  Endereço inválido   
3             Arthur Da Rosa  085.***.***-30     53   NaN  Endereço inválido   
4               Juan Pereira  520.***.***-21     68   NaN  Endereço inválido   
5     Dra. Ana Laura Ribeiro  210.***.***-20     69   NaN  Endereço inválido   
..                       ...             ...    ...   ...                ...   
971            Eloah Peixoto  064.***.***-16     79   NaN  Endereço inválido   
972              Luiza Sales  726.***.***-80     56   NaN  Endereço inválido   
973      Dr. Alexandre Sales  427.***.***-40     87   NaN  Endereço inválido   
974            Alícia Campos  162.***.***-94     49   NaN  Endereço inválido   
975              Luigi Porto  826.***.***-83     43   NaN  Endereço inválido   

    esta

##  Salvar o DataFrame final

In [32]:
df.to_csv('C:/Users/karin/OneDrive/Área de Trabalho/EBAC/clientes_remove_outliers.csv', index=False)